In [1]:
import pandasql as ps
import pandas as pd
import numpy as np
from pyspark import SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
import time
import csv

In [ ]:
##In this script all the dataset are extracted from the database

In [ ]:
spark.sql("SHOW databases").show()
spark.sql("USE covid_2020_q2jun")
spark.sql("show tables").show()

In [ ]:
+--------------------+
|        databaseName|
+--------------------+
|christianacare_in...|
|       covid_2020_q2|
|    covid_2020_q2apr|
|    covid_2020_q2jun|
|             default|
|immersion_day_exa...|
|             pankush|
+--------------------+

+----------------+--------------------+-----------+
|        database|           tableName|isTemporary|
+----------------+--------------------+-----------+
|covid_2020_q2jun|           condition|      false|
|covid_2020_q2jun|          covid_labs|      false|
|covid_2020_q2jun|           demo_temp|      false|
|covid_2020_q2jun|        demographics|      false|
|covid_2020_q2jun|           encounter|      false|
|covid_2020_q2jun|  med_rec_compliance|      false|
|covid_2020_q2jun|          medication|      false|
|covid_2020_q2jun|multum_dnum_refer...|      false|
|covid_2020_q2jun|multum_drug_categ...|      false|
|covid_2020_q2jun|multum_drug_categ...|      false|
|covid_2020_q2jun|njms_covid_demo_e...|      false|
|covid_2020_q2jun|njms_covid_demo_e...|      false|
|covid_2020_q2jun|           njms_demo|      false|
|covid_2020_q2jun|           procedure|      false|
|covid_2020_q2jun|              result|      false|
+----------------+--------------------+-----------+

In [ ]:
#extract the demographic
df_demographics = spark.sql("""SELECT  *  from demographics""").toPandas()

In [ ]:
df_demographics.info()

In [ ]:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117496 entries, 0 to 117495
Data columns (total 7 columns):
tenant       117496 non-null object
personid     117496 non-null object
gender       117496 non-null object
race         117496 non-null object
ethnicity    117496 non-null object
zip_code     117496 non-null object
deceased     117496 non-null int32
dtypes: int32(1), object(6)
memory usage: 5.8+ MB

In [ ]:
df_demographics_selected = df_demographics[['personid', 'gender', 'race', 'ethnicity', 'deceased']]

In [ ]:
df_demographics_selected.to_csv('df_demographics_selected.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
#extract the covid table
df_covid_labs = spark.sql("""SELECT  *  from covid_labs""").toPandas()

In [ ]:
df_covid_labs.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146105 entries, 0 to 146104
Data columns (total 11 columns):
tenant               146105 non-null object
personid             146105 non-null object
encounterid          146105 non-null object
resultid             146105 non-null object
servicedate          146105 non-null object
codetype             146105 non-null object
labcode              146105 non-null object
labtest              146105 non-null object
result               146105 non-null object
pos_cvd19_lab_ind    146105 non-null int32
encountertype        144893 non-null object
dtypes: int32(1), object(10)
memory usage: 11.7+ MB

In [ ]:
df_covid_labs_selected = df_covid_labs[['personid', 'resultid', 'servicedate', 'result', 'pos_cvd19_lab_ind', 'encountertype']] 

In [ ]:
df_covid_labs_selected.to_csv('df_covid_labs_selected.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
df_covid_labs['result'].value_counts()

Negative         100098
Positive          41870
Unknown            3476
Not done            624
Indeterminate        37
Name: result, dtype: int64

In [ ]:
df_covid_labs['pos_cvd19_lab_ind'].value_counts()

0    106667
1     39438
Name: pos_cvd19_lab_ind, dtype: int64

In [ ]:
df_demographicWithCovid= df_demographics_selected.loc[df_demographics_selected['personid'].isin(df_covid_labs['personid'])]

In [ ]:
df_demographicWithCovid.shape
(66455, 5)

In [ ]:
print(df_covid_labs_selected.columns)
print(df_demographics_selected.columns)
df_demographicWithCovid.to_csv('df_demographicWithCovid.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
Index(['personid', 'resultid', 'servicedate', 'result', 'pos_cvd19_lab_ind',
       'encountertype'],
      dtype='object')
Index(['personid', 'gender', 'race', 'ethnicity', 'deceased'], dtype='object')

In [ ]:
test=df_covidPatients_personID[0:10000]

In [ ]:
##updated code for Aaron to Run

In [ ]:
#complete personID with Covid
df_covidPatients_personID.to_csv('df_covidPatients_personID.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
selected_personID = df_covidPatients_personID[1000:2000]

In [ ]:
counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_condition_table_2000 = pd.DataFrame()
for id in selected_personID:
    #print(id)
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas()
    df_covid_from_condition_table_2000 = pd.concat([df_covid_from_condition_table_2000,df_id])
    counter = counter + 1
    if(counter%1000 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))

In [ ]:
df_covid_from_condition_table_2000.to_csv('df_covid_from_condition_table_2000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
##updated code to run for Aaron 11/12/20
selected_personID = df_covidPatients_personID[0:1000]

In [ ]:
#collecting the 1st 1000 from medication, procedure, and result

counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_condition_table_6000 = pd.DataFrame()
df_covid_from_medication_table_6000 = pd.DataFrame()
df_covid_from_procedure_table_6000 = pd.DataFrame()
df_covid_from_result_table_6000 = pd.DataFrame()

for id in selected_personID:
    #print(id)
    
    """
    #table: condition
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas()
    df_covid_from_condition_table_6000 = pd.concat([df_covid_from_condition_table_6000,df_id])
    """
    """
    #table: medication
    query_1 = "SELECT personid, startdate, stopdate, drug, dosequantity, doseunit, route, frequency, asneeded, status from medication WHERE personid='{}'".format(id)
    df_id_1 = sqlContext.sql(query).toPandas()
    df_covid_from_medication_table_6000 = pd.concat([df_covid_from_medication_table_6000,df_id_1])
    """
    
    #table: procedure
    query_2 = "SELECT personid, codetype, procedurecode, procedure, servicestartdate, serviceenddate from procedure WHERE personid='{}' AND year(servicestartdate) = 2020".format(id)
    df_id_2 = sqlContext.sql(query_2).toPandas()
    df_covid_from_procedure_table_6000 = pd.concat([df_covid_from_procedure_table_6000,df_id_2])
    
    #table: result
    query_3 = "SELECT personid, result, textvalue, numericvalue, numericvaluemodifier, unitofmeasure, servicedate from result WHERE personid='{}' AND  year(servicedate) = 2020".format(id)
    df_id_3 = sqlContext.sql(query_3).toPandas()
    df_covid_from_result_table_6000 = pd.concat([df_covid_from_result_table_6000,df_id_3])

    
    
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))

# Save output in csv
#df_covid_from_condition_table_6000.to_csv('df_covid_from_condition_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
#df_covid_from_medication_table_6000.to_csv('df_covid_from_medication_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
df_covid_from_procedure_table_6000.to_csv('df_covid_from_procedure_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
df_covid_from_result_table_6000.to_csv('df_covid_from_result_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
#########################################################################################
#Just Run the above for loop 
#below is the seperated parts

In [ ]:

#collecting the 1st 1000 from medication

counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_medication_table_1000 = pd.DataFrame()
for id in selected_personID:
    #print(id)
    query = "SELECT personid, startdate, stopdate, drug, dosequantity, doseunit, route, frequency, asneeded, status from medication WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas()
    df_covid_from_medication_table_1000 = pd.concat([df_covid_from_medication_table_1000,df_id])
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))
df_covid_from_medication_table_1000.to_csv('df_covid_from_medication_table_1000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
#collecting the 1st 1000 from procedure

counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_procedure_table_1000 = pd.DataFrame()
for id in selected_personID:
    #print(id)
    query = "SELECT personid, codetype, procedurecode, procedure, servicestartdate, serviceenddate from procedure WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas()
    df_covid_from_procedure_table_1000 = pd.concat([df_covid_from_procedure_table_1000,df_id])
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))
df_covid_from_procedure_table_1000.to_csv('df_covid_from_procedure_table_1000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
#collecting the 1st 1000 from result

counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_result_table_1000 = pd.DataFrame()
for id in selected_personID:
    #print(id)
    query = "SELECT personid, codetype, result, textvalue, numericvalue, numericvaluemodifier, unitofmeasure from result WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas()
    df_covid_from_result_table_1000 = pd.concat([df_covid_from_result_table_1000,df_id])
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))
df_covid_from_result_table_1000.to_csv('df_covid_from_result_table_1000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)




In [ ]:
#information extracted from medication table
# df_medication.info()
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 10000 entries, 0 to 9999
# Data columns (total 15 columns):

# personid        10000 non-null object


# startdate       10000 non-null object
# stopdate        10000 non-null object

# drug            10000 non-null object
# dosequantity    8921 non-null object
# doseunit        8916 non-null object
# route           9510 non-null object
# frequency       8516 non-null object
# asneeded        10000 non-null bool
# status          10000 non-null object



# df_procedure.info()
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 692403 entries, 0 to 692402
# Data columns (total 10 columns):

# personid            692403 non-null object


# codetype            692403 non-null object
# procedurecode       692403 non-null object
# procedure           692403 non-null object
# servicestartdate    692403 non-null object
# serviceenddate      692403 non-null object





Remaining tables to be created

In [ ]:

selected_personID=df_covidPatients_personID[3000:3100] #testing for 100 USERS

RequiredResultColumn = ('Respiratory rate', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure',
                        'Non-invasive mean arterial pressure', 'Body temperature', 'Oxygen saturation in Blood',
                        'Inhaled oxygen concentration', 'Inhaled oxygen flow rate', 'Oxygen therapy',
                        'Oxygen/Inspired gas setting [Volume Fraction] Ventilator', 
                        'Cholesterol [Mass/volume] in Serum or Plasma',
                        'Cholesterol.total/Cholesterol in HDL [Mass Ratio] in Serum or Plasma',
                        'Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay',
                        'Triglyceride [Mass/volume] in Body fluid', 
                        'Hemoglobin A1c/Hemoglobin.total in Blood by HPLC',
                        'Protein [Mass/volume] in Body fluid', 'Albumin [Mass/volume] in Urine',
                        'Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma',
                        'Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma',
                        'Alkaline phosphatase.bone [Mass/volume] in Serum or Plasma',
                        'Bilirubin.total [Mass/volume] in Blood',
                        'Bilirubin.direct [Moles/volume] in Serum or Plasma',
                        'Troponin T.cardiac [Mass/volume] in Serum or Plasma',
                        'Troponin I.cardiac [Units/volume] in Serum or Plasma',
                        'Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method',
                        'Natriuretic peptide B [Mass/volume] in Blood',
                        'Urate [Mass/volume] in Urine',    
                        'C reactive protein [Mass/volume] in Serum or Plasma',
                        'INR in Blood by Coagulation assay',
                        'Prothrombin time (PT) in Blood by Coagulation assay',
                        'aPTT in Blood by Coagulation assay', 
                        'Ferritin [Mass/volume] in Serum or Plasma',
                        'Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma',
                        'Fibrin D-dimer FEU [Mass/volume] in Blood by Immunoassay',
                        'Fibrin D-dimer DDU [Mass/volume] in Platelet poor plasma',
                        'Fibrin D-dimer [Units/volume] in Platelet poor plasma',
                        'Leukocytes [#/volume] in Blood',
                        'Neutrophils [#/volume] in Blood',
                        'Lymphocytes [#/volume] in Blood',
                        'Erythrocytes [#/volume] in Blood',
                        'Hematocrit [Volume Fraction] of Blood',
                        'Hemoglobin [Mass/volume] in Blood',
                        'MCH [Entitic mass]', 'MCV [Entitic volume]', 'Platelets [#/volume] in Blood',
                        'Creatinine [Mass/volume] in Blood', 'Urea nitrogen [Mass/volume] in Blood',
                        'Sodium [Moles/volume] in Blood', 
                        'Potassium [Moles/volume] in Blood',
                        'Magnesium [Mass/volume] in Serum or Plasma',
                        'Calcium [Mass/volume] in Serum or Plasma',
                        'Calcium.ionized [Mass/volume] in Blood',
                        '25-Hydroxyvitamin D2+25-Hydroxyvitamin D3 [Mass/volume] in Serum or Plasma',
                        'Ammonia [Mass/volume] in Plasma',
                        'Creatine kinase [Enzymatic activity/volume] in Serum or Plasma',
                        'Influenza virus A Ab [Presence] in Serum by Immunoassay',
                        'Influenza virus B Ab [Presence] in Serum by Immunoassay',
                        'Respiratory syncytial virus Ag [Presence] in Unspecified specimen]')

In [ ]:
#collecting the 1st 100 from medication, procedure, and result

counter = 0
start_time = time.time()
#create an Empty DataFrame object
df_covid_from_condition_table_6000 = pd.DataFrame() #Change Name 
df_covid_from_medication_table_6000 = pd.DataFrame() #Change Name
df_covid_from_procedure_table_6000 = pd.DataFrame() #Change Name
df_covid_from_result_table_6000 = pd.DataFrame() #Change Name

for id in selected_personID:
    #print(id)
    
    """
    #table: condition
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{}'".format(id)
    df_id = sqlContext.sql(query).toPandas() 
    df_covid_from_condition_table_6000 = pd.concat([df_covid_from_condition_table_6000,df_id]) #Change Name
    """
    """
    #table: medication
    query_1 = "SELECT personid, startdate, stopdate, drug, dosequantity, doseunit, route, frequency, asneeded, status from medication WHERE personid='{}'".format(id)
    df_id_1 = sqlContext.sql(query).toPandas()
    df_covid_from_medication_table_6000 = pd.concat([df_covid_from_medication_table_6000,df_id_1]) #Change Name
    """
    """
    #table: procedure
    query_2 = "SELECT personid, codetype, procedurecode, procedure, servicestartdate, serviceenddate from procedure WHERE personid='{}' AND year(servicestartdate) = 2020".format(id)
    df_id_2 = sqlContext.sql(query_2).toPandas()
    df_covid_from_procedure_table_6000 = pd.concat([df_covid_from_procedure_table_6000,df_id_2]) #Change Name
    """
    
    #table: result
    query_3 = "SELECT personid, result, textvalue, numericvalue, numericvaluemodifier, unitofmeasure, servicedate from result WHERE personid='{0}' AND  year(servicedate) = 2020 AND result IN {1}".format(id, RequiredResultColumn)
    df_id_3 = sqlContext.sql(query_3).toPandas()
    df_covid_from_result_table_6000 = pd.concat([df_covid_from_result_table_6000,df_id_3]) #Change Name

    
    
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))

# Save output in csv
#Change Names Here
#df_covid_from_condition_table_6000.to_csv('df_covid_from_condition_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
#df_covid_from_medication_table_6000.to_csv('df_covid_from_medication_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
#df_covid_from_procedure_table_6000.to_csv('df_covid_from_procedure_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
df_covid_from_result_table_6000.to_csv('df_covid_from_result_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
##ICD10 Codes selected by Aaron and Dr. Shakar (Condition Codes)
selected_icd10_codes =('150.1','150.5','150.9','307.81','322.9','339','345',
                       '426','427.5','435.9','518.81','780','780.6','784','785.52','995.9',
                       '995.91','995.92','995.93','995.94','B20','B97.35','E10','E11','E13',
                       'E14','E66.0','E66.09','E66.1','E66.2','E66.3','E66.8','E66.9',
                       'F10.1','F10.2','F10.9','G04.30','G40','G47.3','G47.30','G47.31',
                       'G47.32','G47.33','G47.34','G47.35','G47.36','G47.37','G47.39','G61.0',
                       'G93.40','G93.41','G93.49','I10','I11','I11.0','I11.9','I12.0','I12.9',
                       'I13.0','I13.10','I13.11','I13.2','I15.0','I15.1','I15.2','I15.8',
                       'I15.9','I21.01','I21.02','I21.09','I21.11','I21.19','I21.21','I21.29',
                       'I21.3','I21.4','I22.2','I25.1','I44.0','I44.1','I44.2','I44.30','I44.39',
                       'I46','I46.2','I46.8','I46.9','I48','I48.0','I48.1','I48.11','I48.19',
                       'I48.2','I48.20','I48.21','I48.3','I48.4','I48.9','I48.91','I48.92',
                       'I67.81','I67.82','I67.83','I67.9','I68.0','I68.2','I69.00','I69.01',
                       'I69.010','I69.011','I69.014','I73.9','I74.10','I74.11','I74.19','I74.2',
                       'I74.3','I74.4','I74.5','I74.8','I74.9','I82.210''I82.220','I82.290',
                       'I82.4','I85.01','I85.11','J44','J44.0','J44.1','J44.9','J45.2','J45.20',
                       'J45.21','J45.22','J45.3','J45.30','J45.31','J45.32','J45.40','J45.41',
                       'J45.42','J45.5','J45.50','J45.51','J45.52','J45.901','J45.902','J45.909',
                       'J45.990','J45.991','J45.998','J80','K22.11','K22.6','K29.01','K29.21',
                       'K29.31','K29.41','K29.51','K29.61','K29.71','K29.81','K29.91','K31.811',
                       'K31.82','K50.011','K50.111','K50.811','K50.911','K51.011','K51.211',
                       'K51.311','K51.411','K51.511','K51.811','K51.911','K57.01','K57.11',
                       'K70','K70.0','K70.1','K70.10','K70.11','K70.2','K70.3','K70.30','K70.31',
                       'K70.4','K70.40','K70.41','K70.9','K72.0','K72.00','K72.01','K72.1',
                       'K72.10','K72.11','K72.9','K72.90','K72.91','K74','K74.0','K74.1','K74.2',
                       'K74.3','K74.4','K74.5','K74.6','K74.60','K74.69','K75.8','K75.89','K75.9',
                       'K76.0','K76.1','K76.2','K76.3','K76.4','K76.5','K76.6','K76.7','K76.81',
                       'K76.89','K76.9','K77','K85','K85.0''K85.00','K85.01','K85.02','K85.1',
                       'K85.10','K85.11','K85.12','K85.2','K85.20','K85.21','K85.22','K85.3',
                       'K85.30','K85.31','K85.32','K85.8','K85.80','K85.81','K85.82','K85.9',
                       'K85.90','K85.91','K85.92','K86.0','K86.1','K92.0','K92.1','N17','N17.0',
                       'N17.1','N17.2','N17.8','N17.9','N18','N18.1','N18.2','N18.3','N18.4',
                       'N18.5','N18.6','N18.9','N19','R00.1','R16.0','R17','R45.1','R51','R65.2',
                       'R65.20','R65.21','R74.0','V45.11','V45.12','Z21','Z71.41','Z91.15',
                       'Z92.81','Z94.4','Z99.0','Z99.11','Z99.12','Z99.2','Z99.3','Z99.81','Z99.89')

In [ ]:
##Procedure Codes selected by Aaron and Dr. Shakar (Condition Codes)
selected_procedure_codes = ('43260','43261','43262','43263','43264','43265','43277','43278','43274',
                            '43275','43276','43232','43238','43242','44407','45342','45392','43191',
                            '43193','43197','43198','43200','43202','43235','43239','44360','44361',
                            '44376','44377','44380','44382','44388','44389','45330','45331','45378',
                            '45380','43231','43237','43259','44406','45341','45391','43195','43196',
                            '43213','43214','43220','43226','43233','43245','43248','43249','44381',
                            '44405','45340','45386','43194','43215','43247','44363','44390','45332',
                            '45379','43216','43250','44365','44392','45333','45384','43217','43251',
                            '44364','44394','45338','45385','43229','43270','44369','44401','45346',
                            '45388','43227','43255','44366','44378','44391','45334','45382','43204',
                            '43243','43205','43244','43212','43266','44370','44379','44384','44402',
                            '45347','45389','43246','43762','43763','44372','44373','49450','43192',
                            '43201','43236','44404','45335','45381','43232','43238','43242','44407',
                            '45342','45392')

In [ ]:
##selecting the IDs for extracting the information from the condition table
selected_personID_cond_AND_proc = df_covidPatients_personID[5000:6000] #Change the range manually


##extracting information from the condiiton table
counter = 0
start_time = time.time()

df_covid_from_condition_table_6000 = pd.DataFrame() #Change Name 
df_covid_from_procedure_table_6000 = pd.DataFrame() #Change Name

for id in selected_personID_cond_AND_proc:
    #print(id)
    
    #table: procedure
    query_2 = "SELECT personid, codetype, procedurecode, procedure, servicestartdate, serviceenddate from procedure WHERE personid='{0}' AND year(servicestartdate) = 2020 AND procedurecode IN {1}".format(id, selected_procedure_codes)
    df_id_2 = sqlContext.sql(query_2).toPandas()
    df_covid_from_procedure_table_6000 = pd.concat([df_covid_from_procedure_table_6000,df_id_2]) #Change Name


    #table: condition
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{0}' AND conditioncode IN {1}".format(id, selected_icd10_codes)
    df_id = sqlContext.sql(query).toPandas() 
    df_covid_from_condition_table_6000 = pd.concat([df_covid_from_condition_table_6000,df_id]) #Change Name
    
    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))

##Save output in csv
##Change Names Here
df_covid_from_procedure_table_6000.to_csv('df_covid_from_procedure_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
df_covid_from_condition_table_6000.to_csv('df_covid_from_condition_table_6000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
#################################
##Date: 11/20/20 Code to run for Aaron

In [ ]:
##extracting the condition and results for the FIRST 1000 
extracted_deceased_personIDs = deceased_personIDs_series[0:1000]

##extracting from the result table 
counter = 0
start_time = time.time()

#create an Empty DataFrame object
df_demo_condition_table_deceased_1000 = pd.DataFrame() #Change Name 

for id in extracted_deceased_personIDs:
    #print(id)

    #table: condition
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{0}' AND conditioncode IN {1}".format(id, selected_icd10_codes)
    df_id = sqlContext.sql(query).toPandas() 
    df_demo_condition_table_deceased_1000 = pd.concat([df_demo_condition_table_deceased_1000,df_id]) #Change Name

    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))


# Save output in csv
#Change Names Here
df_demo_condition_table_deceased_1000.to_csv('df_demo_condition_table_deceased_1000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
##extracting the condition and results for the 1000 onwards
extracted_deceased_personIDs = deceased_personIDs_series[1000:2000]

##extracting from the result table 
counter = 0
start_time = time.time()

#create an Empty DataFrame object
df_demo_condition_table_deceased_2000 = pd.DataFrame() #Change Name 
df_demo_result_table_deceased_2000 = pd.DataFrame() #Change Name

for id in extracted_deceased_personIDs:
    #print(id)

    #table: condition
    query = "SELECT personid, conditioncode, condition, classification from condition WHERE personid='{0}' AND conditioncode IN {1}".format(id, selected_icd10_codes)
    df_id = sqlContext.sql(query).toPandas() 
    df_demo_condition_table_deceased_2000 = pd.concat([df_demo_condition_table_deceased_2000,df_id]) #Change Name

    #table: result
    query_3 = "SELECT personid, result, textvalue, numericvalue, numericvaluemodifier, unitofmeasure, servicedate from result WHERE personid='{0}' AND  year(servicedate) = 2020 AND result IN {1}".format(id, RequiredResultColumn)
    df_id_3 = sqlContext.sql(query_3).toPandas()
    df_demo_result_table_deceased_2000 = pd.concat([df_demo_result_table_deceased_2000,df_id_3]) #Change Name

    counter = counter + 1
    if(counter%100 == 0):
        print("Counter Number: {0}".format(counter))
print("Counter Number: {0}".format(counter))
print("Time: {0}".format(time.time()-start_time))


# Save output in csv
#Change Names Here
df_demo_condition_table_deceased_2000.to_csv('df_demo_condition_table_deceased_2000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
df_demo_result_table_deceased_2000.to_csv('df_demo_result_table_deceased_2000.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
